In [94]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
from pandas_datareader import data as pdr
from yahoo_earnings_calendar import YahooEarningsCalendar
import numpy as np

ticker = yf.Ticker("AEB")

start = datetime.datetime(2000,1,1)
end = datetime.datetime(2021,2,28)

print("Start Date: ", start)
print("End Date: ", end)

ticker_daily = ticker.history(interval='1d', start=start, end=end)


Start Date:  2000-01-01 00:00:00
End Date:  2021-02-28 00:00:00


In [95]:
ticker_daily

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-11-23,11.609362,11.609362,11.609362,11.609362,162900,0.0,0
2005-11-25,11.609362,11.609362,11.609362,11.609362,84000,0.0,0
2005-11-28,11.609362,11.679580,11.585956,11.609362,888000,0.0,0
2005-11-29,11.679583,11.679583,11.585958,11.679583,70700,0.0,0
2005-11-30,11.656176,11.679582,11.585958,11.656176,45600,0.0,0
...,...,...,...,...,...,...,...
2021-02-22,25.280001,25.280001,25.129999,25.219999,18600,0.0,0
2021-02-23,25.219999,25.219999,25.070000,25.090000,27200,0.0,0
2021-02-24,25.049999,25.180000,25.049999,25.049999,12000,0.0,0


In [96]:
ticker.info

IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
msft_daily.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits'],
      dtype='object')

In [89]:
ticker_name = 'ABEV'
ticker = yf.Ticker(ticker_name)
ticker_data = ticker.history(interval='1d', start=start, end=end)
ticker_data['ticker'] = ticker_name
ticker_data.index.name = 'market_date'
ticker_data.reset_index(inplace=True)

new_cols = ['market_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'dividends', 'stock_splits', 'ticker']

ticker_data.columns = new_cols

ticker_data['year_month'] = ticker_data.market_date.astype(str).str[:7]

ticker_data['daily_price_change'] = ticker_data.close_price - ticker_data.close_price.shift(1)
ticker_data['daily_return'] = ticker_data.close_price / np.where(ticker_data.close_price.shift(1) == 0, .01, ticker_data.close_price.shift(1)) - 1
ticker_data['volume_change'] = ticker_data.volume - ticker_data.volume.shift(1)
ticker_data['volume_perc_change'] = ticker_data.volume / np.where(ticker_data.volume.shift(1) == 0, .01, ticker_data.volume.shift(1)) - 1


def f(x):
    d = {}
    d['lowest_close'] = x['close_price'].min()
    d['highest_close'] = x['close_price'].max()
    d['avg_close'] = x['close_price'].mean()
    d['lowest_open'] = x['open_price'].min()
    d['highest_open'] = x['open_price'].max()
    d['avg_open'] = x['open_price'].mean()
    d['highest_high'] = x['high_price'].max()
    d['lowest_low'] = x['low_price'].min()
    d['daily_return_volatility'] = x['daily_return'].std()
    d['avg_daily_return'] = x['daily_return'].mean()
    d['monthly_return'] = (x['daily_return'] + 1).prod()
    d['total_price_change'] = x['daily_price_change'].sum()
    d['daily_volume_volatility'] = x['volume_perc_change'].std()
    d['avg_volume_change'] = x['volume_perc_change'].mean()
    d['monthly_perc_volume_change'] = (x['volume_perc_change'] + 1).prod()
    d['total_volume_change'] = x['volume_perc_change'].sum()
    
    return pd.Series(d, index=['lowest_close', 'highest_close', 'avg_close', 'lowest_open', 'highest_open','avg_open',
                              'highest_high', 'lowest_low', 'daily_return_volatility', 'avg_daily_return', 
                               'monthly_return','total_price_change', 'daily_volume_volatility', 'avg_volume_change', 
                               'monthly_perc_volume_change','total_volume_change'])

ticker_data_monthly = ticker_data.groupby(['ticker', 'year_month']).apply(f)

In [93]:
x = pd.nan
x

AttributeError: module 'pandas' has no attribute 'nan'

In [90]:
test = ticker_data_monthly.reset_index()


In [91]:
ticker_data[ticker_data.market_date < '2000-02-01']

,market_date,open_price,high_price,low_price,close_price,volume,dividends,stock_splits,ticker,year_month,daily_price_change,daily_return,volume_change,volume_perc_change
0,2000-01-03,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,NaN,NaN,NaN,NaN
1,2000-01-04,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0
2,2000-01-05,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0
3,2000-01-06,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0
4,2000-01-07,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0
5,2000-01-10,0.187767,0.187767,0.187767,0.187767,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0
6,2000-01-11,0.184311,0.184311,0.179704,0.179704,30000,0.0,0.0,ABEV,2000-01,-0.008063,-0.042944,30000.0,2999999.0
7,2000-01-12,0.179704,0.179704,0.179704,0.179704,30000,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,0.0
8,2000-01-13,0.179704,0.179704,0.179704,0.179704,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,-30000.0,-1.0
9,2000-01-14,0.179704,0.179704,0.179704,0.179704,0,0.0,0.0,ABEV,2000-01,0.000000,0.000000,0.0,-1.0


In [7]:
msft_daily[msft_daily.Dividends > 0]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
784,2003-02-19,15.815128,15.853359,15.400953,15.630342,46902700,0.08,0.0
950,2003-10-15,18.710421,18.748867,18.390038,18.627121,57242100,0.16,0.0
1164,2004-08-23,17.525281,17.563840,17.428882,17.506001,39572200,0.08,0.0
1223,2004-11-15,19.582782,19.697385,19.482505,19.618595,104468000,3.08,0.0
1286,2005-02-15,18.680432,18.737910,18.579845,18.630138,76551600,0.08,0.0
...,...,...,...,...,...,...,...,...
5063,2020-02-19,186.168133,186.286921,184.594132,185.395981,29997500,0.51,0.0
5127,2020-05-20,183.460370,184.492784,182.596728,184.304169,31261300,0.51,0.0
5190,2020-08-19,210.453046,211.060056,208.224023,208.671814,27627600,0.51,0.0
5254,2020-11-18,213.159057,214.675569,210.445306,210.594971,28372800,0.56,0.0


In [8]:
msft.dividends

Date
2003-02-19    0.08
2003-10-15    0.16
2004-08-23    0.08
2004-11-15    3.08
2005-02-15    0.08
              ... 
2020-02-19    0.51
2020-05-20    0.51
2020-08-19    0.51
2020-11-18    0.56
2021-02-17    0.56
Name: Dividends, Length: 69, dtype: float64

In [13]:
msft.earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [14]:
msft.quarterly_balance_sheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [15]:
msft.quarterly_cashflow

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [63]:
ticker_actions = msft.actions
ticker_actions.reset_index(inplace=True)

ticker_actions.columns = ['market_date', 'dividends', 'stock_splits']

ticker_actions['ticker'] = 'msft'

ticker_actions = ticker_actions[['ticker', 'market_date', 'dividends', 'stock_splits']]

ticker_actions

,ticker,market_date,dividends,stock_splits
0,msft,2003-02-18,0.00,2.0
1,msft,2003-02-19,0.08,0.0
2,msft,2003-10-15,0.16,0.0
3,msft,2004-08-23,0.08,0.0
4,msft,2004-11-15,3.08,0.0
...,...,...,...,...
65,msft,2020-02-19,0.51,0.0
66,msft,2020-05-20,0.51,0.0
67,msft,2020-08-19,0.51,0.0
68,msft,2020-11-18,0.56,0.0


In [18]:
type(msft)

yfinance.ticker.Ticker

In [19]:
type(msft.earnings)

pandas.core.frame.DataFrame

In [32]:
msft.get_financials()
msft.get_major_holders()

,0,1
0,0.06%,% of Shares Held by All Insider
1,71.84%,% of Shares Held by Institutions
2,71.89%,% of Float Held by Institutions
3,4849,Number of Institutions Holding Shares


In [35]:
msft.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'previousClose', 'regularMarketOpen', 'twoHundredDayAverage', 'trailingAnnualDividendYield', 'payoutRatio', 'volume24Hr', 'regularMarketDayHigh', 'navPrice', 'averageDailyVolume10Day', 'totalAssets', 'regularMarketPreviousClose', 'fiftyDayAverage', 'trailingAnnualDividendRate', 'open', 'toCurrency', 'averageVolume10days', 'expireDate', 'yield', 'algorithm', 'dividendRate', 'exDividendDate', 'beta', 'circulatingSupply', 'startDate', 'regularMarketDayLow', 'priceHint', 'currency', 'trailingPE', 'regularMarketVolume', 'lastMarket', 'maxSupply', 'openInterest', 'marketCap', 'volumeAllCurrencies', 'strikePrice', 'averageVolume', 'priceToSalesTrailing12Months', 'dayLow', 'ask', 'ytdReturn', 'askSize', 'volume', 'fiftyTwoWeekHigh', 'forwardPE', 'fromCurrency', 'fiveYearAvgDividendYield', 'fiftyTwoWeekLow', 'bid', 'tradeabl

In [66]:
msft_info = {key: msft.info[key] for key in [
    'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'state', 'website'
]}
msft_info

ticker_info = msft.info
desired_cols = ['sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'state', 'website']
ticker_info = {key: ticker_info[key] for key in desired_cols}

ticker_info_df = pd.DataFrame(ticker_info, index=[0])

ticker_info_df


,sector,fullTimeEmployees,longBusinessSummary,city,state,website
0,Technology,163000,"Microsoft Corporation develops, licenses, and ...",Redmond,WA,http://www.microsoft.com


In [22]:
from yahoo_earnings_calendar import YahooEarningsCalendar

In [30]:
yec = YahooEarningsCalendar()
earnings_list = yec.get_earnings_of(symbol='MSFT')
earnings_df = pd.DataFrame(earnings_list)

In [32]:
earnings_df

,ticker,companyshortname,startdatetime,startdatetimetype,epsestimate,epsactual,epssurprisepct,timeZoneShortName,gmtOffsetMilliSeconds,quoteType
0,MSFT,Microsoft Corp,2022-01-24T10:59:00.000Z,TNS,NaN,NaN,NaN,EST,-18000000,EQUITY
1,MSFT,Microsoft Corp,2021-10-25T10:59:00.000Z,TNS,NaN,NaN,NaN,EST,-18000000,EQUITY
2,MSFT,Microsoft Corp,2021-07-20T10:59:00.000Z,TNS,NaN,NaN,NaN,EST,-18000000,EQUITY
3,MSFT,Microsoft Corp,2021-04-27T20:00:00.000Z,AMC,1.77,NaN,NaN,EST,-18000000,EQUITY
4,MSFT,Microsoft Corporation,2021-01-26T00:00:00.000Z,TNS,1.64,2.03,23.63,EST,0,EQUITY
...,...,...,...,...,...,...,...,...,...,...
95,MSFT,Microsoft Corporation,1998-04-22T00:00:00.000Z,TNS,0.12,0.13,5.72,EST,0,EQUITY
96,MSFT,Microsoft Corporation,1998-01-21T00:00:00.000Z,TNS,0.10,0.11,3.85,EST,0,EQUITY
97,MSFT,Microsoft Corporation,1997-10-20T00:00:00.000Z,TNS,0.09,0.09,2.53,EST,0,EQUITY
98,MSFT,Microsoft Corporation,1997-07-17T00:00:00.000Z,TNS,0.10,0.10,0.83,EST,0,EQUITY


In [28]:
1.4 / 1.26

1.111111111111111

In [29]:
1.51/1.32

1.1439393939393938

In [100]:
x = 'MSFT'

x_dict = {'x': x}

temp_df = pd.DataFrame(x_dict, index = [0])

In [101]:
temp_df

,x
0,MSFT
